# Generate trades using dedicated service - Python

### Overview
Retrieving trades over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request trades and illustrate market activity over a look back period.

<div class="alert alert-block alert-info">
    <b>Note:</b> To highlight individual trades among market activity. Please refer to <i>bestex-pointintime</i> and <i>bestex-interval</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades over the time.

### Inputs/outputs
Trades extraction sample requires instrument's identifier, date time intervals as per inputs and returns the occured trades information.

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matpotlib* as per display packages
    
***

# Run Trades sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_trades_pb2 as tick_trades
import systemathics.apis.services.tick.v1.tick_trades_pb2_grpc as tick_trades_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Retrieve data
To request *tick trades* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick trades request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'GLE'
exchange = 'XPAR'

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 9, day = 13), 
    end_date = date.Date(year = 2021, month = 9, day = 13)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval1 = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 6, minutes = 50, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 7, minutes = 15, seconds = 0)
)
time_interval2 = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 15, minutes = 20, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 15, minutes = 45, seconds = 0)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
constraint = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval1,time_interval2],
)

In [ ]:
# generate the tick trades request
id = identifier.Identifier(exchange = exchange, ticker = ticker)
request = tick_trades.TickTradesRequest(
    identifiers = [id],
    constraints = constraint
)

In [ ]:
# open a gRPC channel
with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
    credentials = grpc.ssl_channel_credentials(f.read())
with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:
    
    # instantiate the tick trades service
    service = tick_trades_service.TickTradesServiceStub(channel)
    
    # process the tick trades request
    trades = []
    metadata = [('authorization', token)]
    for trade in service.TickTrades(request=request, metadata=metadata):
        if trade.trade:
            trades.append(trade.trade)

# keep only trades, delete instrument mapping (1st element retrieved)
# mapping is not used as only one instrument identifier is requested
del trades[0]

The following code snippet displays the total retrieved trades count:

In [ ]:
print('Total trades retrieved: {0}'.format(len(trades)))

### Step 4: Visualize data

#### 4.1 Retrieve data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# prepare the dataframe content
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in trades]
prices = [t.price for t in trades]
sizes = [t.size for t in trades]
ids = [t.id for t in trades]
conditions = [t.condition for t in trades]

In [ ]:
# create a pandas dataframe with: dates, trades prices and sizes
d = {'Date': dates, 'Price': prices, 'Size': sizes, 'Ids': ids, 'Conditions': conditions}
df = pd.DataFrame(data=d)
df

In [ ]:
filename = 'trades{0}-{1}.csv'.format(ticker, exchange)
df.to_csv(filename)
df[df['Conditions'] == '2']